In [1]:
import pandas as pd

In [2]:
chatbotData = pd.read_csv("ChatData.csv", encoding='utf-8')
question, answer = list(chatbotData['Q']), list(chatbotData['A'])

In [3]:
question = question[:100]
answer = answer[:100]

In [4]:
for i in range(10):
    print("질문:", question[i])
    print("답변:", answer[i])
    print()

질문: 12시 땡!
답변: 하루가 또 가네요.

질문: 1지망 학교 떨어졌어
답변: 위로해 드립니다.

질문: 3박4일 놀러가고 싶다
답변: 여행은 언제나 좋죠.

질문: 3박4일 정도 놀러가고 싶다
답변: 여행은 언제나 좋죠.

질문: PPL 심하네
답변: 눈살이 찌푸려지죠.

질문: SD카드 망가졌어
답변: 다시 새로 사는 게 마음 편해요.

질문: SD카드 안돼
답변: 다시 새로 사는 게 마음 편해요.

질문: SNS 맞팔 왜 안하지ㅠㅠ
답변: 잘 모르고 있을 수도 있어요.

질문: SNS 시간낭비인 거 아는데 매일 하는 중
답변: 시간을 정하고 해보세요.

질문: SNS 시간낭비인데 자꾸 보게됨
답변: 시간을 정하고 해보세요.



In [5]:
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from konlpy.tag import Okt
from keras import models, layers, optimizers, metrics, preprocessing

Using TensorFlow backend.


In [6]:
#태그 단어

PAD = "<PADDING>" #패딩
STA = "<START>" #시작
END = "<END>" #끝
OOV = "<OOV>" #out of vocabulary

PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_TARGET = 2

#한 문장에서 단어 시퀀스의 최대 개수
maxSequences = 30

#임베딩 벡터 차원
embeddingDim = 100

#LSTM 출력 차원
lstmHiddenDim = 128

#정규표현식 필터
RE_FILTER = re.compile("[.,!?\"'~()]")

In [7]:
#형태소 분석기
def posTag(sentences):
    
    tagger = Okt()
    
    sentencePos=[]
    for sentence in sentences:
        #특수문자 제거
        sentence = re.sub(RE_FILTER, "", sentence) #sentence에 있는 글자 중에서 RE_FILTER에 해당하는 글자 제거
        sentence = " ".join(tagger.morphs(sentence))
        sentencePos.append(sentence)
        
    return sentencePos

In [8]:
question = posTag(question)
answer =posTag(answer)

In [9]:
question
answer

['하루 가 또 가네요',
 '위로 해 드립니다',
 '여행 은 언제나 좋죠',
 '여행 은 언제나 좋죠',
 '눈살 이 찌푸려지죠',
 '다시 새로 사는 게 마음 편해요',
 '다시 새로 사는 게 마음 편해요',
 '잘 모르고 있을 수도 있어요',
 '시간 을 정 하고 해보세요',
 '시간 을 정 하고 해보세요',
 '자랑 하는 자리 니까 요',
 '그 사람 도 그럴 거 예요',
 '그 사람 도 그럴 거 예요',
 '혼자 를 즐기세요',
 '돈 은 다시 들어올 거 예요',
 '땀 을 식혀주세요',
 '어서 잊고 새 출발 하세요',
 '빨리 집 에 돌아가서 끄고 나오세요',
 '빨리 집 에 돌아가서 끄고 나오세요',
 '다음 달 에는 더 절약 해봐요',
 '따뜻하게 사세요',
 '다음 달 에는 더 절약 해봐요',
 '가장 확실한 시간 은 오늘이 에요 어제 와 내일 을 놓고 고민 하느라 시간 을 낭비하지 마세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '더 가까워질 기회 가 되겠네요',
 '저 도 요',
 '다 들 바빠서 이야기 할 시간 이 부족했나 봐요',
 '다 들 바빠서 이야기 할 시간 이 부족했나 봐요',
 '온 가족 이 모두 마음 에 드는 곳 으로 가보세요',
 '좋은 생각 이에요',
 '더 가까워질 기회 가 되겠네요',
 '저 를 만들어 준 사람 을 부모님 저 랑 이야기 해 주는 사람 을 친구 로 생각 하고 있어요',
 '좋은 생각 이에요',
 '정말 후회 할 습관 이에요',
 '무모한 결정 을 내 리지 마세요',
 '선생님 이나 기관 에 연락 해보세요',
 '떨리는 감정 은 그 자체 로 소중해요',
 '득템 했길 바라요

In [10]:
#질문 + 대답 문장을 하나로 합치기
sentences=[]
sentences.extend(question) #append는 리스트로 추가
sentences.extend(answer)
len(sentences)

200

In [11]:
#단어 배열 생성
words=[]
for sentence in sentences:
    for word in sentence.split():
        words.append(word)
len(words) #전체 단어의 개수 966개

966

In [12]:
#words에서 길이가 0인 단어를 삭제
words = [word for word in words if len(word)>0]
#중복 단어 삭제
words = list(set(words))

In [13]:
len(words)

450

In [14]:
words[:0] = [PAD, STA, END, OOV]

In [15]:
len(words)

454

In [16]:
words[0]

'<PADDING>'

In [17]:
len(words)
words[:20]

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '싫어하지',
 '자랑',
 '자꾸',
 '고고',
 '아름다운',
 '박',
 '볼까',
 '해주세요',
 '있어',
 '예요',
 '해보여',
 '낮잠',
 '나온거',
 '수도',
 '땀',
 '걸리겠어']

In [18]:
#단어에 대한 인덱스를 부여 -> 딕셔너리
wordToIndex = {word:index for index, word in enumerate(words)}
indexToWord = {index:word for index, word in enumerate(words)}

In [19]:
#전처리
#문장 -> 인덱스로 변환

In [20]:
#                  question, wordToIndex, ENCODER_INPUT
def convertTextToIndex(sentences, voc, mytype):
    
    sentencesIndex=[]
    for sentence in sentences:
        
        sentenceIndex=[]
        if mytype==DECODER_INPUT:
            sentenceIndex.extend([voc[STA]]) #1
            
        for word in sentence.split():
            if voc.get(word) is not None:
                sentenceIndex.extend([voc[word]]) #단어에 해당되는 인덱스가 추가
                
            else: #사전에 없는 단어의 경우 OOV 추가
                sentenceIndex.extend([voc[OOV]])                    
                
        if mytype==DECODER_TARGET: #디코더 출력은 맨 마지막에 end 추가
            if maxSequences <= len(sentenceIndex):
                sentenceIndex = sentenceIndex[:maxSequences-1]+ [voc[END]]
            else:
                sentenceIndex+=[voc[END]]
                
        else:
            if len(sentenceIndex) > maxSequences:
                sentenceIndex = sentenceIndex[:maxSequences]
        #0으로 채움
        sentenceIndex+=[wordToIndex[PAD]]*(maxSequences-len(sentenceIndex))
        sentencesIndex.append(sentenceIndex)
        
    return np.asarray(sentencesIndex)

In [21]:
#인코더 입력, 디코더 입력, 디코더 출력 -> 인덱스 변환
xEncoder = convertTextToIndex(question, wordToIndex, ENCODER_INPUT)
print(xEncoder[0])

xDecoder = convertTextToIndex(answer, wordToIndex, DECODER_INPUT)
print(xDecoder[0])

yDecoder = convertTextToIndex(answer, wordToIndex, DECODER_TARGET)
print(yDecoder[0])

[413  52   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[  1 124 424 192  32   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[124 424 192  32   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [22]:
# 인코더 입력: 12시 땡
# 디코더 입력: START 하루 가  또      가네요
# 디코더 출력: 하루  가   또  가네요  END

In [23]:
# np.zeros((2,3,4))
oneHotData = np.zeros((len(yDecoder), maxSequences, len(words)))
#100(답변개수), 30(최대 단어 개수), 454(전체 단어 집합 개수)

In [24]:
np.shape(oneHotData)  #(100, 30, 454) 디코더 출력
# 하루  가   또  가네요  END
yDecoder.shape  #(100, 30)
for i,seq in enumerate(yDecoder):
    for j,index in enumerate(seq):
        oneHotData[i,j,index]=1
yDecoder = oneHotData

In [25]:
yDecoder[0].shape #첫번째 답변 내용의 shape

(30, 454)

In [26]:
# 모델 생성
# 인코더 정의
# 입력 문장의 인덱스 sequence를 입력
encoderInputs = layers.Input(shape=(None,))
#임베딩 계층
encoderOutputs = layers.Embedding(len(words), embeddingDim)(encoderInputs)

encoderOutputs, stateH, stateC = layers.LSTM(lstmHiddenDim, return_state=True, dropout=0.2, recurrent_dropout=0.5)(encoderOutputs)

#return_state=True  => 상태값 리턴
#LSTM은 2개 상태 존재(cell, hidden state)

encoderStates = [stateH, stateC]


# 디코더 정의
# 출력 문장의 인덱스 sequence를 입력
decoderInputs = layers.Input(shape=(None,))
#임베딩 계층
decoderEmbedding = layers.Embedding(len(words), embeddingDim )
decoderOutputs = decoderEmbedding(decoderInputs)

decoderLSTM = layers.LSTM(lstmHiddenDim, return_state=True, return_sequences=True, dropout=0.2, recurrent_dropout=0.5)
decoderOutputs, _, _ = decoderLSTM(decoderOutputs, initial_state=encoderStates)

decoderDense = layers.Dense(len(words), activation='softmax')
decoderOutputs = decoderDense(decoderOutputs)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [27]:
model = models.Model([encoderInputs,decoderInputs], decoderOutputs)

In [28]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
#예측 모델 인코더 정의
encoderModel = models.Model(encoderInputs, encoderStates)

#예측 모델 디코더 정의
#바로 앞에 있는 디코더의 출력(상태)을 입력 받아서 예측을 해야 함
decoderStateInputH = layers.Input(shape=(lstmHiddenDim,))
decoderStateInputC = layers.Input(shape=(lstmHiddenDim,))
decoderStatesInputs = [decoderStateInputH, decoderStateInputC]

#임베딩 계층
decoderOutputs = decoderEmbedding(decoderInputs)
#LSTM 계층
decoderOutputs, stateH, stateC = decoderLSTM(decoderOutputs, initial_state=decoderStatesInputs)
decoderStates = [stateH, stateC]

#Dense계층을 통해 원핫 형식으로 예측 단어 인덱스를 추출
decoderOutputs= decoderDense(decoderOutputs)

#예측 모델 디코더 설정
decoderModel = models.Model([decoderInputs]+decoderStatesInputs, [decoderOutputs]+decoderStates) 


In [30]:
#인덱스를 문장으로 변환
def convertIndextToText(indexs, voc):
    sentence=''
    for i in indexs:
        if i==END_INDEX: #종료 인덱스
            break
        if voc.get(i) is not None:
            sentence+=voc[i]
        else:
            sentence.extend([voc[OOV_INDEX]])
        sentence+=" "
    
    return sentence

In [33]:
#에폭
for epoch in range(20):
    print("total epoch:", epoch+1)
    history = model.fit([xEncoder, xDecoder], yDecoder, epochs=100, batch_size=64, verbose=0)
    print("accuracy:", history.history['accuracy'])
    print("loss:", history.history['loss'])
    #문장 예측
    #3박 4일 놀러 가고 싶다 -> 여행은 언제나 좋죠
    
    inputEncoder = xEncoder[2].reshape(1, xEncoder[2].shape[0]) #(30,) => (1,30)
    inputDecoder = xDecoder[2].reshape(1, xDecoder[2].shape[0]) #(30,) => (1,30)
    
    results = model.predict([inputEncoder, inputDecoder])
    
    #결과값에 대해서 가장 큰 값의 위치를 구함
    index = np.argmax(results[0], 1)
    #인덱스 -> 문장으로 변환
    sentence = convertIndextToText(index, indexToWord)
    print(sentence)
    print()

total epoch: 1
accuracy: [0.9033333, 0.90566665, 0.904, 0.9036667, 0.907, 0.9066667, 0.907, 0.9063333, 0.90966666, 0.9066667, 0.91033334, 0.9116667, 0.9123333, 0.91033334, 0.911, 0.915, 0.91533333, 0.9123333, 0.91433334, 0.915, 0.9166667, 0.914, 0.917, 0.916, 0.9173333, 0.91833335, 0.917, 0.92266667, 0.9213333, 0.9206667, 0.921, 0.92433333, 0.9216667, 0.92433333, 0.923, 0.925, 0.92733335, 0.925, 0.932, 0.931, 0.9303333, 0.93266666, 0.93133336, 0.934, 0.93666667, 0.933, 0.93766665, 0.932, 0.932, 0.936, 0.93766665, 0.93733335, 0.938, 0.93866664, 0.938, 0.94233334, 0.943, 0.941, 0.9406667, 0.9406667, 0.93833333, 0.945, 0.9446667, 0.945, 0.9446667, 0.9433333, 0.946, 0.9403333, 0.945, 0.9486667, 0.94633335, 0.94766665, 0.949, 0.9493333, 0.948, 0.948, 0.95066667, 0.952, 0.9483333, 0.952, 0.9493333, 0.95166665, 0.949, 0.95133334, 0.95166665, 0.95233333, 0.95166665, 0.955, 0.95133334, 0.9533333, 0.9546667, 0.9543333, 0.953, 0.9543333, 0.956, 0.956, 0.9576667, 0.95566666, 0.95566666, 0.95666665

accuracy: [0.973, 0.973, 0.9713333, 0.971, 0.96966666, 0.971, 0.97, 0.9723333, 0.9713333, 0.973, 0.97033334, 0.97, 0.97033334, 0.97066665, 0.971, 0.97066665, 0.97033334, 0.972, 0.9723333, 0.971, 0.972, 0.9726667, 0.9716667, 0.971, 0.9713333, 0.9713333, 0.9713333, 0.972, 0.9713333, 0.97, 0.9713333, 0.972, 0.9713333, 0.972, 0.9723333, 0.972, 0.9716667, 0.9723333, 0.972, 0.9716667, 0.9716667, 0.9713333, 0.97066665, 0.971, 0.972, 0.972, 0.971, 0.972, 0.9723333, 0.972, 0.9713333, 0.97, 0.9716667, 0.9713333, 0.9716667, 0.9723333, 0.9713333, 0.9723333, 0.971, 0.9726667, 0.9723333, 0.9716667, 0.9716667, 0.969, 0.9716667, 0.9716667, 0.97333336, 0.972, 0.9716667, 0.9726667, 0.9716667, 0.9723333, 0.9726667, 0.9716667, 0.97066665, 0.972, 0.972, 0.97066665, 0.9726667, 0.9726667, 0.973, 0.9726667, 0.9723333, 0.97066665, 0.9713333, 0.972, 0.9716667, 0.972, 0.972, 0.9726667, 0.971, 0.973, 0.973, 0.9726667, 0.9716667, 0.9713333, 0.9713333, 0.9723333, 0.9716667, 0.9713333]
loss: [0.10228588134050369, 0.

accuracy: [0.9766667, 0.9763333, 0.97433335, 0.97333336, 0.97466666, 0.97533333, 0.97533333, 0.97366667, 0.97366667, 0.97566664, 0.9723333, 0.97333336, 0.9726667, 0.974, 0.975, 0.973, 0.976, 0.975, 0.9726667, 0.972, 0.973, 0.9726667, 0.97333336, 0.97333336, 0.975, 0.97333336, 0.97433335, 0.9723333, 0.973, 0.97333336, 0.97333336, 0.975, 0.97366667, 0.97333336, 0.974, 0.97333336, 0.97466666, 0.976, 0.97433335, 0.97366667, 0.975, 0.97566664, 0.97466666, 0.97466666, 0.97433335, 0.97533333, 0.975, 0.97366667, 0.9726667, 0.974, 0.973, 0.97533333, 0.97466666, 0.975, 0.97566664, 0.975, 0.97533333, 0.97433335, 0.97366667, 0.975, 0.97433335, 0.97466666, 0.97566664, 0.974, 0.97333336, 0.97466666, 0.9726667, 0.97466666, 0.975, 0.9763333, 0.97566664, 0.97566664, 0.97566664, 0.973, 0.973, 0.974, 0.976, 0.976, 0.974, 0.97366667, 0.97533333, 0.97566664, 0.974, 0.97466666, 0.97466666, 0.97433335, 0.9726667, 0.97533333, 0.97533333, 0.977, 0.975, 0.97533333, 0.976, 0.97466666, 0.976, 0.97566664, 0.976, 0

accuracy: [0.97533333, 0.97866666, 0.978, 0.977, 0.97433335, 0.974, 0.977, 0.977, 0.9776667, 0.97833335, 0.9776667, 0.9763333, 0.9766667, 0.977, 0.977, 0.97533333, 0.97566664, 0.9773333, 0.9773333, 0.9776667, 0.97566664, 0.97566664, 0.9773333, 0.9776667, 0.976, 0.9763333, 0.97833335, 0.9776667, 0.9776667, 0.97533333, 0.978, 0.97833335, 0.97566664, 0.976, 0.9763333, 0.9766667, 0.976, 0.978, 0.9776667, 0.979, 0.97866666, 0.97833335, 0.97866666, 0.97866666, 0.9773333, 0.9766667, 0.976, 0.9773333, 0.97833335, 0.97366667, 0.979, 0.9776667, 0.978, 0.979, 0.9776667, 0.977, 0.97833335, 0.97833335, 0.97833335, 0.97933334, 0.97833335, 0.9766667, 0.97866666, 0.9773333, 0.97866666, 0.9776667, 0.978, 0.9773333, 0.9766667, 0.9773333, 0.978, 0.9766667, 0.97833335, 0.97866666, 0.9776667, 0.97866666, 0.97933334, 0.979, 0.97966665, 0.97833335, 0.9773333, 0.97833335, 0.9776667, 0.97833335, 0.9803333, 0.9776667, 0.979, 0.977, 0.9766667, 0.977, 0.97866666, 0.97866666, 0.97966665, 0.97933334, 0.97833335, 0.

accuracy: [0.9816667, 0.98466665, 0.982, 0.98233336, 0.982, 0.983, 0.98433334, 0.98233336, 0.982, 0.9826667, 0.9816667, 0.983, 0.98366666, 0.98366666, 0.984, 0.983, 0.9813333, 0.981, 0.983, 0.98333335, 0.9826667, 0.98, 0.982, 0.984, 0.9826667, 0.98333335, 0.98, 0.983, 0.983, 0.98333335, 0.98233336, 0.98466665, 0.984, 0.98333335, 0.98233336, 0.981, 0.97966665, 0.983, 0.982, 0.98366666, 0.9826667, 0.9853333, 0.985, 0.98233336, 0.98233336, 0.982, 0.9826667, 0.9813333, 0.98433334, 0.9853333, 0.985, 0.98333335, 0.98366666, 0.9816667, 0.98366666, 0.98433334, 0.9813333, 0.9826667, 0.982, 0.9816667, 0.983, 0.9816667, 0.98333335, 0.984, 0.9856667, 0.983, 0.98433334, 0.98333335, 0.9863333, 0.981, 0.98366666, 0.98433334, 0.98433334, 0.986, 0.98333335, 0.98, 0.9826667, 0.9826667, 0.985, 0.98433334, 0.98366666, 0.98233336, 0.985, 0.98333335, 0.985, 0.98433334, 0.98366666, 0.9853333, 0.98433334, 0.984, 0.9853333, 0.985, 0.98466665, 0.98433334, 0.984, 0.985, 0.98233336, 0.986, 0.983, 0.984]
loss: [0.

KeyboardInterrupt: 

In [ ]:
# "3박4일 놀러가고 싶다"  #여행은 언제나 좋죠
# "3박4일 같이 놀러가고 싶다"  #여행은 언제나 좋죠
# '4박5일 놀러가고 싶다' ???
# '3박4일 동안 동해로 놀러가고 싶다' ???
# '3박4일 동안 동해로 놀러가려고'  ???

In [34]:
#문장 입력 
#[[320, 157, ....19, 0,0,0,....0]]
def makePredictInput(sentence):
    sentences=[]
    sentences.append(sentence)
    sentences = posTag(sentences)
    inputSeq = convertTextToIndex(sentences, wordToIndex, ENCODER_INPUT)
    return inputSeq

In [35]:
#에측 답변 생성
def generateText(inputSeq):
    #입력을 인코더에 넣고 마지막 상태 구함
    states = encoderModel.predict(inputSeq)
    
    #목표 시퀀스 초기화
    targetSeq = np.zeros((1,1)) 
    #<START> 시그널을 추가
    targetSeq[0,0] = STA_INDEX
    #인덱스 초기화
    indexs=[]
    
    #디코더 반복
    while 1:
        decoderOutputs, stateH, stateC = decoderModel.predict([targetSeq]+states)
        #결과를 원핫인코딩 형식으로 변환
        index = np.argmax(decoderOutputs[0,0,:])
        indexs.append(index)
        #종료 체크
        if index==END_INDEX or len(indexs) >= maxSequences:
            break
        #targetSeq를 이전 출력으로 설정
        targetSeq = np.zeros((1,1)) 
        targetSeq[0,0] = index
        
        #디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [stateH, stateC]
        
    #인덱스를 문장으로 변환
    sentence = convertIndextToText(indexs, indexToWord) 
    
    return sentence

In [36]:
#입력문장 -> 인덱스
inputSeq = makePredictInput("3박4일 같이 놀러가고 싶다")
#[[320, 157, ....19, 0,0,0,....0]]

In [37]:
sentence = generateText(inputSeq)
sentence #여행은 언제나 좋죠

'여행 은 언제나 좋죠 '